In [0]:
!pip install comet_ml

In [0]:
from comet_ml import Experiment

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from sklearn.metrics import accuracy_score
print(torch.__version__)

## Fashion MINST dataset


In [0]:
from torchvision.datasets import FashionMNIST

In [0]:
# Getting the train and test parts of the dataset
data_train = FashionMNIST("FashionMNIST/",
                          download=True,
                          train=True)

data_test = FashionMNIST("FashionMNIST/",
                          download=True,
                          train=False)

# In fact, it's already stored as torch tensor, but we'll need
# to work with the numpy representation, so let's do the convertion:
X_train = data_train.train_data.numpy()
y_train = data_train.train_labels.numpy()

X_test = data_test.test_data.numpy()
y_test = data_test.test_labels.numpy()


The datasets consists of images belonging to one out of 10 classes:

| Label | Description | | Label | Description |
| ---       
| 0        | T-shirt/top   || 5        | Sandal         |
| 1        | Trouser        || 6        | Shirt             |
| 2        | Pullover       || 7        | Sneaker       |
| 3        | Dress           || 8        | Bag              |
| 4        | Coat             || 9        | Ankle boot  |






### Input preprocessing

So far our data is held as numpy arrays of unsigned byte type, i.e. it lies within a range from 0 to 255. Also, the shape of our input is 3-dimensional (num_images, height, width), while our `model` takes 2-dimensional "arrays of 1-dimensional images" (num_images, height * width).

We have to convert that to `torch` tensors and reshape the input. Also, it's a good idea to normalize your image data to lie within a $[0, 1]$ interval. Let's write a function that does all these things:

In [0]:
# Write a function to convert X and y to torch tensors while
# rescaling X to fit into [0, 1] interval and reshaping it properly

# Hint: make sure your input tensor dtype is same as the
# parameters of the model (should be torch.float)

def preprocess_data(X, y):
  X_preprocessed = torch.reshape(torch.from_numpy(X/ 255.).type(torch.float32), (-1, 784))
  y_preprocessed = torch.from_numpy(y)
  return X_preprocessed, y_preprocessed

### Some utilities

In [0]:
# Batch generator
# (here's a very brief description of what python generators are:
# https://stackoverflow.com/a/231855/3801744)
def get_batches(X, y, batch_size, shuffle=False):
  if shuffle:
    shuffle_ids = np.random.permutation(len(X))
    X = X[shuffle_ids].copy()
    y = y[shuffle_ids].copy()
  for i_picture in range(0, len(X), batch_size):
    # Get batch and preprocess it:
    batch_X = X[i_picture:i_picture + batch_size]
    batch_y = y[i_picture:i_picture + batch_size]
    
    # 'return' the batch (see the link above to
    # better understand what 'yield' does)
    yield preprocess_data(batch_X, batch_y)

In [0]:
def get_test_predictions(model, batch_size=100):
  predictions_test = np.concatenate([
    model(batch_X).to('cpu').detach().numpy()
    for batch_X, batch_y in get_batches(X_test, y_test, batch_size)
  ], axis=0)
  return np.argmax(predictions_test, axis=1)


## Main training loop


In [0]:
def train2score(n_epochs, batch_size=100, learning_rate=0.001, hidden_size=100):
  global experiment
  experiment.log_parameters({
      'n_epochs': n_epochs,
      'batch_size': batch_size,
      'learning_rate': learning_rate,
      'hidden_size': hidden_size})
  # Defining the loss function:
  loss_function = torch.nn.CrossEntropyLoss()

  # Defining the model
  input_size = 28 * 28 # number of pixels per image
  output_size = 10 # number of classes

  model = torch.nn.Sequential(
    torch.nn.Linear(input_size, hidden_size),
    torch.nn.ELU(),
    torch.nn.Linear(hidden_size, output_size),
  )

  # Setting up the optimizer
  learning_rate = 0.005
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  for i_epoch in range(n_epochs):
    for batch_X, batch_y in get_batches(X_train, y_train,
                                        batch_size=batch_size,
                                        shuffle=True):

      # Compute the loss, zero the gradients, and make an optimization step
      predictions = model(batch_X) # compute the predictions
      loss = loss_function(predictions, batch_y) # compute the loss

      model.zero_grad() # zero the gradients
      loss.backward() # compute new gradients
      optimizer.step() # do an optimization step

    for batch_X, batch_y in get_batches(X_test, y_test,
                                        batch_size=batch_size):

      # Compute the loss
      predictions = model(batch_X) # compute the predictions
      loss = loss_function(predictions, batch_y) # compute the loss
    accuracy = accuracy_score(get_test_predictions(model), y_test)
    experiment.log_metrics({'accuracy': accuracy, 'loss': loss.item()})

  experiment.end()
  return accuracy

In [0]:
n_epochs = 5
learning_rate = 0.0005
hidden_size = 100
experiment = Experiment(api_key=<YOUR KEY>, project_name="comet demo")
score = train2score(n_epochs=n_epochs, learning_rate=learning_rate, 
                    hidden_size=hidden_size)
print ("Score: {}".format(score))

## Homework

1. Add more convolutional magic

```
model = torch.nn.Sequential(
            Reshape(-1, 1, 28, 28),
            torch.nn.Conv2d(in_channels=1,
                            out_channels=8,
                            kernel_size=3,
                            padding=1),
            torch.nn.Dropout(p=dropout_rate),
            torch.nn.ELU(),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(in_channels=8,
                            out_channels=16,
                            kernel_size=3,
                            padding=1),
            torch.nn.Dropout(p=dropout_rate),
            torch.nn.ELU(),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(in_channels=16,
                            out_channels=32,
                            kernel_size=3,
                            padding=1),
            torch.nn.Dropout(p=dropout_rate),
            torch.nn.ELU(),
            torch.nn.MaxPool2d(2, padding=1),
            Reshape(-1, 512),
            torch.nn.Linear(512, 10)
        ).to(device)
```

2. Optimise it via scikit-optimize package